In [1]:
import gensim
import pandas as pd

C:\Users\Abhay\Anaconda3\envs\toxic-comments\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
train_data_path = '../data/raw_data/train.csv'
test_data_path = '../data/raw_data/test.csv'

In [8]:
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [9]:
train_data.shape
test_data.shape

(63978, 2)

In [16]:
comments = list(train_data['Comment'])
comments[0]

"Explanation\r\r\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [17]:
preprocessed_comments = []
for i, line in enumerate(comments):
    preprocessed_comments.append(gensim.utils.simple_preprocess(line))

In [18]:
preprocessed_comments[0]

['explanation',
 'why',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'hardcore',
 'metallica',
 'fan',
 'were',
 'reverted',
 'they',
 'weren',
 'vandalisms',
 'just',
 'closure',
 'on',
 'some',
 'gas',
 'after',
 'voted',
 'at',
 'new',
 'york',
 'dolls',
 'fac',
 'and',
 'please',
 'don',
 'remove',
 'the',
 'template',
 'from',
 'the',
 'talk',
 'page',
 'since',
 'retired',
 'now']

In [24]:
num_features = 300 # The dimension of the word vector
min_word_count = 3# The minimum word count
num_of_workers = 4 # Number of threads to be used in parallel
context = 10 # The context window size
downsampling = 1e-3

In [25]:
model = gensim.models.Word2Vec(
        preprocessed_comments,
        size = num_features,
        window = context,
        min_count = min_word_count,
        workers = num_of_workers,
        sample = downsampling)

In [26]:
model.train(preprocessed_comments, total_examples = len(comments), epochs=10)

(77612448, 101876800)

In [27]:
# Vocabulary size
len(model.wv.vocab)

59701

In [23]:
w1 = "hey"
model.wv.most_similar(positive=w1)

[('hi', 0.6357199549674988),
 ('hello', 0.5268846750259399),
 ('yo', 0.472095251083374),
 ('howdy', 0.4563082456588745),
 ('dude', 0.45139196515083313),
 ('sup', 0.4460800886154175),
 ('haha', 0.4426163136959076),
 ('mate', 0.4378395080566406),
 ('btw', 0.4340026080608368),
 ('hahahaha', 0.42958885431289673)]